In [1]:
import json
import numpy as np
import pandas as pd
import requests

from pathlib import Path

In [2]:
path = Path('images/boxart')
path.mkdir(parents=True, exist_ok=True)

In [3]:
def resolve(filename):
    return 'https://cdn.thegamesdb.net/images/original/' + filename

In [4]:
def save(game, url):
    image = requests.get(url)
    filename = f"{game}.png"
    
    location = path.joinpath(filename)
    
    with open(location, 'wb') as handle:
        if image.status_code == 200:
            handle.write(image.content)
        else:
            print('Error')

In [5]:
def get_games(file):
    games = file.get('data').get('games')
    dataframe = pd.DataFrame(games)

    # SNES: 6, NES:  7

    mask = (
        (dataframe.platform == 6) |
        (dataframe.platform == 7)
    )

    drop = [
        'alternates', 
        'coop', 
        'hashes', 
        'players', 
        'uids', 
        'youtube'
    ]

    dataframe = dataframe[mask]
    dataframe = dataframe.drop(drop, axis=1)

    return dataframe

In [6]:
def get_boxart(file):
    boxart = file.get('include').get('boxart')
    _ = boxart.get('base_url').get('original')
    images = boxart.get('data')

    boxart = []

    for key, value in images.items():
        for v in value:
            condition = (
                v.get('type') == 'boxart' and
                v.get('side') == 'front'
            )

            if condition:
                v['id'] = int(key)
                boxart.append(v)

    drop = [
        'side',
        'type'
    ]

    dataframe = pd.DataFrame(boxart)
    dataframe = dataframe.drop(drop, axis=1)

    return dataframe

In [7]:
with open('data/database.json', 'r') as handle:
    file = json.load(handle)

In [8]:
games = get_games(file)

In [9]:
games

,id,game_title,release_date,platform,region_id,country_id,overview,rating,developers,genres,publishers
3,5,Donkey Kong,1986-06-01,7,2,50,Can you save Mario&#039;s girl from the clutch...,E - Everyone,[6053],[15],[3]
35,41,Super Mario Kart,1992-09-01,6,0,0,The Super Mario GoKart Park is open for tons o...,E - Everyone,[6037],[7],[3]
70,76,Bionic Commando,1988-12-21,7,0,0,Bionic Commando takes place ten years after an...,E - Everyone,[1436],"[1, 15]",[9]
77,83,Final Fantasy III,1994-10-11,6,0,0,MagiTek has been reborn. And the end of the wo...,T - Teen,[8096],[4],[11]
106,112,Super Mario Bros. 3,1990-02-12,7,0,0,"Fight monsters and mini-bosses, avoid ghosts a...",E - Everyone,[6055],[15],[3]
...,...,...,...,...,...,...,...,...,...,...,...
91746,104463,Famicom 198 Jeux CoolBoy,2020-01-01,7,4,28,198 Famicom Games -- Pirate,E - Everyone,None,[28],None
91747,104464,Famicom 400 Jeux CoolBoy,2020-01-01,7,4,28,400 Famicom games -- Pirate,E - Everyone,None,[28],None
91823,104549,Lennus: Kodai Kikai no Kioku,1992-11-13,6,4,28,"Paladin&#039;s Quest, originally released as L...",Not Rated,[1823],[4],[314]
91841,104568,Retroplayer.nl Championship 2015,1970-01-01,7,9,33,Retroplayer.nl Championship 2015 is a competit...,Not Rated,None,"[1, 5, 7, 15]",None


In [10]:
boxart = get_boxart(file)

In [11]:
boxart

,id,filename,resolution
0,2,boxart/front/2-1.jpg,1525x2160
1,10,boxart/front/10-1.jpg,None
2,11,boxart/front/11-1.jpg,640x904
3,12,boxart/front/12-1.jpg,None
4,13,boxart/front/13-1.jpg,800x1135
...,...,...,...
90689,104694,boxart/front/104694-1.jpg,None
90690,104695,boxart/front/104695-1.jpg,None
90691,104696,boxart/front/104696-1.jpg,None
90692,104697,boxart/front/104697-1.jpg,None


In [12]:
dataframe = games.merge(
    boxart, 
    how='inner', 
    on='id'
)

dataframe

,id,game_title,release_date,platform,region_id,country_id,overview,rating,developers,genres,publishers,filename,resolution
0,5,Donkey Kong,1986-06-01,7,2,50,Can you save Mario&#039;s girl from the clutch...,E - Everyone,[6053],[15],[3],boxart/front/5-2.jpg,1534x2100
1,41,Super Mario Kart,1992-09-01,6,0,0,The Super Mario GoKart Park is open for tons o...,E - Everyone,[6037],[7],[3],boxart/front/41-1.jpg,2100x1534
2,76,Bionic Commando,1988-12-21,7,0,0,Bionic Commando takes place ten years after an...,E - Everyone,[1436],"[1, 15]",[9],boxart/front/76-1.jpg,None
3,83,Final Fantasy III,1994-10-11,6,0,0,MagiTek has been reborn. And the end of the wo...,T - Teen,[8096],[4],[11],boxart/front/83-1.jpg,2100x1536
4,112,Super Mario Bros. 3,1990-02-12,7,0,0,"Fight monsters and mini-bosses, avoid ghosts a...",E - Everyone,[6055],[15],[3],boxart/front/112-2.jpg,1534x2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,104463,Famicom 198 Jeux CoolBoy,2020-01-01,7,4,28,198 Famicom Games -- Pirate,E - Everyone,None,[28],None,boxart/front/104463-1.jpg,None
4622,104464,Famicom 400 Jeux CoolBoy,2020-01-01,7,4,28,400 Famicom games -- Pirate,E - Everyone,None,[28],None,boxart/front/104464-1.jpg,None
4623,104549,Lennus: Kodai Kikai no Kioku,1992-11-13,6,4,28,"Paladin&#039;s Quest, originally released as L...",Not Rated,[1823],[4],[314],boxart/front/104549-1.jpg,None
4624,104568,Retroplayer.nl Championship 2015,1970-01-01,7,9,33,Retroplayer.nl Championship 2015 is a competit...,Not Rated,None,"[1, 5, 7, 15]",None,boxart/front/104568-1.jpg,None


In [13]:
filename = dataframe.filename.to_numpy()

url = np.frompyfunc(
    lambda x: resolve(x),
    1,
    1
)(filename)

dataframe['url'] = url

In [14]:
dataframe

,id,game_title,release_date,platform,region_id,country_id,overview,rating,developers,genres,publishers,filename,resolution,url
0,5,Donkey Kong,1986-06-01,7,2,50,Can you save Mario&#039;s girl from the clutch...,E - Everyone,[6053],[15],[3],boxart/front/5-2.jpg,1534x2100,https://cdn.thegamesdb.net/images/original/box...
1,41,Super Mario Kart,1992-09-01,6,0,0,The Super Mario GoKart Park is open for tons o...,E - Everyone,[6037],[7],[3],boxart/front/41-1.jpg,2100x1534,https://cdn.thegamesdb.net/images/original/box...
2,76,Bionic Commando,1988-12-21,7,0,0,Bionic Commando takes place ten years after an...,E - Everyone,[1436],"[1, 15]",[9],boxart/front/76-1.jpg,None,https://cdn.thegamesdb.net/images/original/box...
3,83,Final Fantasy III,1994-10-11,6,0,0,MagiTek has been reborn. And the end of the wo...,T - Teen,[8096],[4],[11],boxart/front/83-1.jpg,2100x1536,https://cdn.thegamesdb.net/images/original/box...
4,112,Super Mario Bros. 3,1990-02-12,7,0,0,"Fight monsters and mini-bosses, avoid ghosts a...",E - Everyone,[6055],[15],[3],boxart/front/112-2.jpg,1534x2100,https://cdn.thegamesdb.net/images/original/box...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,104463,Famicom 198 Jeux CoolBoy,2020-01-01,7,4,28,198 Famicom Games -- Pirate,E - Everyone,None,[28],None,boxart/front/104463-1.jpg,None,https://cdn.thegamesdb.net/images/original/box...
4622,104464,Famicom 400 Jeux CoolBoy,2020-01-01,7,4,28,400 Famicom games -- Pirate,E - Everyone,None,[28],None,boxart/front/104464-1.jpg,None,https://cdn.thegamesdb.net/images/original/box...
4623,104549,Lennus: Kodai Kikai no Kioku,1992-11-13,6,4,28,"Paladin&#039;s Quest, originally released as L...",Not Rated,[1823],[4],[314],boxart/front/104549-1.jpg,None,https://cdn.thegamesdb.net/images/original/box...
4624,104568,Retroplayer.nl Championship 2015,1970-01-01,7,9,33,Retroplayer.nl Championship 2015 is a competit...,Not Rated,None,"[1, 5, 7, 15]",None,boxart/front/104568-1.jpg,None,https://cdn.thegamesdb.net/images/original/box...


In [15]:
column = ['id', 'url']
collection = dataframe[column].to_numpy().transpose()
game, url = collection

url = np.frompyfunc(
    lambda x, y: save(x, y),
    2,
    0
)(game[:2], url[:2])

In [16]:
# # Search
# search = [
#     0
# ]

# search = bockbluster[bockbluster.country_id.isin(search)]
# search.head(5)